In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist, fashion_mnist


In [ ]:
# ===== CONFIGURATION VARIABLES =====
dataset_choice = "mnist"        # "mnist" or "fashion"
epochs = 50                     # recommended 30–100
batch_size = 128                # 64 or 128
noise_dim = 100                 # 50 or 100
learning_rate = 0.0002
save_interval = 5


In [ ]:
if dataset_choice == "mnist":
    (x_train, y_train), _ = mnist.load_data()
elif dataset_choice == "fashion":
    (x_train, y_train), _ = fashion_mnist.load_data()
else:
    raise ValueError("Invalid dataset choice")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Normalize images to [-1, 1]
x_train = (x_train.astype("float32") - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)

img_shape = x_train.shape[1:]


In [ ]:
os.makedirs("generated_samples", exist_ok=True)
os.makedirs("final_generated_images", exist_ok=True)


In [ ]:
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(256, input_dim=noise_dim),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(),

        layers.Dense(512),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(),

        layers.Dense(28 * 28, activation='tanh'),
        layers.Reshape((28, 28, 1))
    ])
    return model


In [ ]:
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=img_shape),
        layers.Dense(512),
        layers.LeakyReLU(0.2),

        layers.Dense(256),
        layers.LeakyReLU(0.2),

        layers.Dense(1, activation='sigmoid')
    ])
    return model


In [ ]:
generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate, 0.5),
    metrics=["accuracy"]
)

discriminator.trainable = False

z = layers.Input(shape=(noise_dim,))
fake_img = generator(z)
validity = discriminator(fake_img)

gan = tf.keras.Model(z, validity)
gan.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate, 0.5)
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
def save_images(epoch):
    noise = np.random.normal(0, 1, (25, noise_dim))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(5, 5, figsize=(5,5))
    idx = 0
    for i in range(5):
        for j in range(5):
            axs[i,j].imshow(gen_imgs[idx,:,:,0], cmap='gray')
            axs[i,j].axis('off')
            idx += 1

    fig.savefig(f"generated_samples/epoch_{epoch:02d}.png")
    plt.close()


In [ ]:
half_batch = batch_size // 2

for epoch in range(1, epochs + 1):

    idx = np.random.randint(0, x_train.shape[0], half_batch)
    real_imgs = x_train[idx]

    noise = np.random.normal(0, 1, (half_batch, noise_dim))
    fake_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((half_batch, 1)))

    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    valid_labels = np.ones((batch_size, 1))

    g_loss = gan.train_on_batch(noise, valid_labels)

    print(f"Epoch {epoch}/{epochs} | D_loss: {d_loss[0]:.2f} | "
          f"D_acc: {d_loss[1]*100:.2f}% | G_loss: {g_loss:.2f}")

    if epoch % save_interval == 0:
        save_images(epoch)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step 


/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/trainer.py:83: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 1/50 | D_loss: 0.66 | D_acc: 58.20% | G_loss: 0.59
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Epoch 2/50 | D_loss: 0.70 | D_acc: 46.48% | G_loss: 0.58
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Epoch 3/50 | D_loss: 0.71 | D_acc: 43.52% | G_loss: 0.57
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Epoch 4/50 | D_loss: 0.70 | D_acc: 43.78% | G_loss: 0.56
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Epoch 5/50 | D_loss: 0.71 | D_acc: 42.34% | G_loss: 0.55
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Epoch 6/50 | D_loss: 0.71 | D_acc: 42.31% | G_loss: 0.54
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Epoch 7/50 | D_loss: 0.71 | D_acc: 41.77% | G_loss: 0.52
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Epoch 8/50 | D_loss: 0.72 | D_acc: 40.76% | G_loss: 0.51
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Epoch 9/50 | D_loss: 0.72 | D_acc: 40.29% | G_loss: 0.50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Epoch 10/50 | D_loss: 0.72 | D_acc: 39.97% | G_loss: 0.49
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
2/2 ━━━━━━

In [ ]:
noise = np.random.normal(0, 1, (100, noise_dim))
final_images = generator.predict(noise)
final_images = 0.5 * final_images + 0.5

for i in range(100):
    plt.imshow(final_images[i,:,:,0], cmap='gray')
    plt.axis('off')
    plt.savefig(f"final_generated_images/img_{i}.png")
    plt.close()


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step


In [ ]:
classifier = tf.keras.Sequential([
    layers.Flatten(input_shape=(28,28,1)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

classifier.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

classifier.fit(x_train, y_train, epochs=3, batch_size=256, verbose=0)

pred_labels = classifier.predict(final_images)
pred_labels = np.argmax(pred_labels, axis=1)

unique, counts = np.unique(pred_labels, return_counts=True)
label_distribution = dict(zip(unique, counts))

print("\nLabel Distribution of Generated Images:")
print(label_distribution)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step

Label Distribution of Generated Images:
{np.int64(6): np.int64(84), np.int64(7): np.int64(16)}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ===== GOOGLE DRIVE BASE PATH =====
BASE_DIR = "/content/drive/MyDrive/Lab_2_GAN"

MODEL_DIR = os.path.join(BASE_DIR, "models")
SAMPLE_DIR = os.path.join(BASE_DIR, "generated_samples")
FINAL_DIR = os.path.join(BASE_DIR, "final_generated_images")
LOG_DIR = os.path.join(BASE_DIR, "logs")

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(SAMPLE_DIR, exist_ok=True)
os.makedirs(FINAL_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)


In [ ]:
def save_images(epoch):
    noise = np.random.normal(0, 1, (25, noise_dim))
    gen_imgs = generator.predict(noise, verbose=0)
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(5, 5, figsize=(5,5))
    idx = 0
    for i in range(5):
        for j in range(5):
            axs[i,j].imshow(gen_imgs[idx,:,:,0], cmap='gray')
            axs[i,j].axis('off')
            idx += 1

    fig.savefig(os.path.join(SAMPLE_DIR, f"epoch_{epoch:02d}.png"))
    plt.close()


In [ ]:
log_file = open(os.path.join(LOG_DIR, "training_log.txt"), "w")


In [ ]:
log_msg = (f"Epoch {epoch}/{epochs} | "
           f"D_loss: {d_loss[0]:.2f} | "
           f"D_acc: {d_loss[1]*100:.2f}% | "
           f"G_loss: {g_loss:.2f}")

print(log_msg)
log_file.write(log_msg + "\n")


Epoch 50/50 | D_loss: 0.84 | D_acc: 33.54% | G_loss: 0.26


58

In [ ]:
log_file.close()


In [ ]:
noise = np.random.normal(0, 1, (100, noise_dim))
final_images = generator.predict(noise, verbose=0)
final_images = 0.5 * final_images + 0.5

for i in range(100):
    plt.imshow(final_images[i,:,:,0], cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(FINAL_DIR, f"img_{i}.png"))
    plt.close()
